In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('mel_features_dataset.csv')

In [3]:
X = df['Mel_Features'].values
y = df['Ragas'].values

In [4]:
X = np.array(df['Mel_Features'].tolist())
y = np.array(df['Ragas'].tolist())

In [5]:
print("Shape of X before reshape:", X.shape)
print("Shape of y before encode:", y.shape)

Shape of X before reshape: (92,)
Shape of y before encode: (92,)


In [6]:
le=LabelEncoder()
ti=le.fit_transform(y)
y_mel=to_categorical(ti)

In [7]:
y_mel

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y_mel,test_size=0.2,random_state=127)

In [9]:
num_classes = len(y_mel[0])
print(num_classes)

3


In [10]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train_flat = np.array([x.flatten() for x in X_train])

In [11]:
print("Shape of X_train:", X_train_flat.shape)

Shape of X_train: (73, 1)


In [12]:
y_train_numerical = np.argmax(y_train, axis=1)
y_test_numerical = np.argmax(y_test, axis=1)
y_train_encoded = to_categorical(y_train_numerical, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_numerical, num_classes=num_classes)

In [13]:
print(X_train[0])

['[[-53.29912567 -37.78041077 -27.55327797 ... -80.         -80.\n  -80.        ]\n [-54.55303574 -36.44094086 -27.5207653  ... -80.         -80.\n  -80.        ]\n [-55.10224152 -36.41986847 -29.47444153 ... -80.         -80.\n  -80.        ]\n ...\n [-53.42518234 -37.04436111 -24.7610302  ... -80.         -80.\n  -80.        ]\n [-57.92776489 -36.66670227 -24.26868057 ... -80.         -80.\n  -80.        ]\n [-33.79752731 -30.21833801 -21.74354172 ... -80.         -80.\n  -80.        ]]']


In [14]:
print(X[0])

[[-61.36026764 -56.3211441  -39.66576004 ... -80.         -80.
  -80.        ]
 [-62.71931458 -53.63751221 -39.98358154 ... -80.         -80.
  -80.        ]
 [-60.93008041 -51.95860291 -40.66412354 ... -80.         -80.
  -80.        ]
 ...
 [-62.52468491 -56.15249634 -40.87715149 ... -77.84117126 -80.
  -80.        ]
 [-47.95288467 -45.6871109  -39.36861038 ... -79.81661224 -80.
  -80.        ]
 [-37.45179749 -36.0095253  -33.8965683  ... -77.08987427 -78.45914459
  -80.        ]]


In [32]:
import ast
import re

list_of_arrays = []
for i, item in enumerate(X_train):
    print(f"\n--- Processing item {i} ---")
    print(f"Original item (type {type(item)}): '{item}'")

    try:
        # Step 1: Evaluate the outermost string literal
        evaluated_list = ast.literal_eval(str(item))
        print(f"After ast.literal_eval: '{evaluated_list}' (type {type(evaluated_list)})")

        # Check if the evaluated_list is indeed a list and has at least one element
        if isinstance(evaluated_list, list) and len(evaluated_list) > 0:
            # Get the actual string containing the array data from inside the list
            evaluated_item_str = evaluated_list[0]
            print(f"Extracted string from list: '{evaluated_item_str}' (type {type(evaluated_item_str)})")


        # Step 2: Clean up the evaluated string
        cleaned_string = evaluated_item_str.strip('[]').replace('\n', ' ')
        print(f"After strip/replace newlines: '{cleaned_string}'")

        # Step 3: Handle multiple spaces and final strip
        cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()
        print(f"After regex and final strip: '{cleaned_string}'")

        if cleaned_string:
            # Step 4: Convert to 1D array
            temp_array = np.fromstring(cleaned_string, sep=' ')
            print(f"Parsed temp_array shape: {temp_array.shape}, data: {temp_array}")
            list_of_arrays.append(temp_array) # Appending 1D array for now

        else:
            print("Cleaned string was empty. Appending empty NumPy array.")
            list_of_arrays.append(np.array([]))

    except (ValueError, SyntaxError) as e:
        print(f"!!! ERROR processing item {i}: '{item}' - Error: {e}")
        print("Appending empty NumPy array due to error.")
        list_of_arrays.append(np.array([]))
    except Exception as e: # Catch any other unexpected errors
        print(f"!!! UNEXPECTED ERROR processing item {i}: '{item}' - Error: {e}")
        print("Appending empty NumPy array due to unexpected error.")
        list_of_arrays.append(np.array([]))


--- Processing item 0 ---
Original item (type <class 'numpy.ndarray'>): '['[[-53.29912567 -37.78041077 -27.55327797 ... -80.         -80.\n  -80.        ]\n [-54.55303574 -36.44094086 -27.5207653  ... -80.         -80.\n  -80.        ]\n [-55.10224152 -36.41986847 -29.47444153 ... -80.         -80.\n  -80.        ]\n ...\n [-53.42518234 -37.04436111 -24.7610302  ... -80.         -80.\n  -80.        ]\n [-57.92776489 -36.66670227 -24.26868057 ... -80.         -80.\n  -80.        ]\n [-33.79752731 -30.21833801 -21.74354172 ... -80.         -80.\n  -80.        ]]']'
After ast.literal_eval: '['[[-53.29912567 -37.78041077 -27.55327797 ... -80.         -80.\n  -80.        ]\n [-54.55303574 -36.44094086 -27.5207653  ... -80.         -80.\n  -80.        ]\n [-55.10224152 -36.41986847 -29.47444153 ... -80.         -80.\n  -80.        ]\n ...\n [-53.42518234 -37.04436111 -24.7610302  ... -80.         -80.\n  -80.        ]\n [-57.92776489 -36.66670227 -24.26868057 ... -80.         -80.\n  -80.  

<ipython-input-32-aad8703c5529>:28: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  temp_array = np.fromstring(cleaned_string, sep=' ')


In [36]:
list_of_test_arrays = []
for i, item in enumerate(X_test):
    print(f"\n--- Processing item {i} ---")
    print(f"Original item (type {type(item)}): '{item}'")

    try:
        # Step 1: Evaluate the outermost string literal
        evaluated_list = ast.literal_eval(str(item))
        print(f"After ast.literal_eval: '{evaluated_list}' (type {type(evaluated_list)})")

        # Check if the evaluated_list is indeed a list and has at least one element
        if isinstance(evaluated_list, list) and len(evaluated_list) > 0:
            # Get the actual string containing the array data from inside the list
            evaluated_item_str = evaluated_list[0]
            print(f"Extracted string from list: '{evaluated_item_str}' (type {type(evaluated_item_str)})")


        # Step 2: Clean up the evaluated string
        cleaned_string = evaluated_item_str.strip('[]').replace('\n', ' ')
        print(f"After strip/replace newlines: '{cleaned_string}'")

        # Step 3: Handle multiple spaces and final strip
        cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()
        print(f"After regex and final strip: '{cleaned_string}'")

        if cleaned_string:
            # Step 4: Convert to 1D array
            temp_array = np.fromstring(cleaned_string, sep=' ')
            print(f"Parsed temp_array shape: {temp_array.shape}, data: {temp_array}")
            list_of_test_arrays.append(temp_array) # Appending 1D array for now

        else:
            print("Cleaned string was empty. Appending empty NumPy array.")
            list_of_test_arrays.append(np.array([]))

    except (ValueError, SyntaxError) as e:
        print(f"!!! ERROR processing item {i}: '{item}' - Error: {e}")
        print("Appending empty NumPy array due to error.")
        list_of_test_arrays.append(np.array([]))
    except Exception as e: # Catch any other unexpected errors
        print(f"!!! UNEXPECTED ERROR processing item {i}: '{item}' - Error: {e}")
        print("Appending empty NumPy array due to unexpected error.")
        list_of_test_arrays.append(np.array([]))


--- Processing item 0 ---
Original item (type <class 'numpy.ndarray'>): '['[[-52.37485123 -47.80406952 -47.22372437 ... -69.48109436 -66.16337585\n  -80.        ]\n [-49.3175354  -48.95485306 -47.37657928 ... -66.26097107 -64.6263504\n  -80.        ]\n [-37.82212067 -38.91293716 -43.93315125 ... -56.79515839 -64.38362122\n  -78.98735046]\n ...\n [-57.3351059  -51.57400513 -61.18727112 ... -75.87779236 -78.69697571\n  -80.        ]\n [-59.63704681 -52.29603577 -59.16051865 ... -77.93856049 -80.\n  -80.        ]\n [-56.24648285 -51.70728683 -49.69155121 ... -75.66121674 -76.82054901\n  -80.        ]]']'
After ast.literal_eval: '['[[-52.37485123 -47.80406952 -47.22372437 ... -69.48109436 -66.16337585\n  -80.        ]\n [-49.3175354  -48.95485306 -47.37657928 ... -66.26097107 -64.6263504\n  -80.        ]\n [-37.82212067 -38.91293716 -43.93315125 ... -56.79515839 -64.38362122\n  -78.98735046]\n ...\n [-57.3351059  -51.57400513 -61.18727112 ... -75.87779236 -78.69697571\n  -80.        ]\n [

<ipython-input-36-775f896da4f8>:28: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  temp_array = np.fromstring(cleaned_string, sep=' ')


In [37]:
print("\n--- Final list_of_arrays status ---")
print(f"Total number of arrays parsed: {len(list_of_arrays)}")
if len(list_of_arrays) > 0:
    print(f"Shape of the first parsed array: {list_of_arrays[0].shape}")
    print(f"First parsed array content: {list_of_arrays[0]}")
    try:
        final_dataset = np.array(list_of_arrays)
        print(f"Shape of the final combined dataset: {final_dataset.shape}")
    except ValueError as e:
        print(f"Cannot combine list_of_arrays into a single NumPy array due to inconsistent shapes: {e}")


--- Final list_of_arrays status ---
Total number of arrays parsed: 73
Shape of the first parsed array: (3,)
First parsed array content: [-53.29912567 -37.78041077 -27.55327797]
Shape of the final combined dataset: (73, 3)


In [38]:
print("\n--- Final list_of_test_arrays status ---")
print(f"Total number of arrays parsed: {len(list_of_test_arrays)}")
if len(list_of_test_arrays) > 0:
    print(f"Shape of the first parsed array: {list_of_test_arrays[0].shape}")
    print(f"First parsed array content: {list_of_test_arrays[0]}")
    try:
        final_test_dataset = np.array(list_of_test_arrays)
        print(f"Shape of the final combined dataset: {final_test_dataset.shape}")
    except ValueError as e:
        print(f"Cannot combine list_of_arrays into a single NumPy array due to inconsistent shapes: {e}")


--- Final list_of_test_arrays status ---
Total number of arrays parsed: 19
Shape of the first parsed array: (3,)
First parsed array content: [-52.37485123 -47.80406952 -47.22372437]
Shape of the final combined dataset: (19, 3)


In [34]:
model_mel = Sequential()
model_mel.add(Conv1D(32, 3, activation='relu', input_shape=(final_dataset.shape[0], 1)))
model_mel.add(MaxPooling1D(2))
model_mel.add(Conv1D(64, 3, activation='relu'))
model_mel.add(MaxPooling1D(2))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
model_mel = Sequential()
model_mel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(final_dataset.shape[1], final_dataset.shape[2], 1)))
model_mel.add(MaxPooling2D((2, 2)))
model_mel.add(Conv2D(64, (3, 3), activation='relu'))
model_mel.add(MaxPooling2D((2, 2)))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

IndexError: tuple index out of range

In [35]:
model_mel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_mel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 71, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 35, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 33, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 16, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 145,987 (570.26 KB)

 Trainable params: 145,987 (570.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#print("X_train dtype:", X_train_float.dtype)
#print("y_train_encoded dtype:", y_train_encoded.dtype)
#print("X_test dtype:", X_test_float.dtype)
#print("y_test_encoded dtype:", y_test_encoded.dtype)

In [39]:
history_mel = model_mel.fit(final_dataset, y_train_encoded, epochs=30, batch_size=32, validation_data=(final_test_dataset, y_test_encoded))

Epoch 1/30


ValueError: Exception encountered when calling MaxPooling1D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_1/max_pooling1d_1/MaxPool1d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 1, 2, 1], padding="VALID", strides=[1, 1, 2, 1]](sequential_1/max_pooling1d_1/MaxPool1d/ExpandDims)' with input shapes: [?,1,1,32].[0m

Arguments received by MaxPooling1D.call():
  • inputs=tf.Tensor(shape=(None, 1, 32), dtype=float32)